In [0]:
import json
import requests
import pandas as pd
import time
import numpy as np

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Stats scraper

In [0]:
# grabs data from stats.nba.com and stores in dataframe
# WARNING: VERY very VERY slow.
# Future improvement: run multiple instances. Containers? Mapreduce?

game_ids = {}
for year in range(2009, 2020):
  for month in range(1, 13):
    for day in range(1, 32):
      month_ = ("{:02d}".format(month))
      day_ = ("{:02d}".format(day))
      print("Getting game_ids from " + month_ + "/" + day_ + "/" + str(year))
      url = "https://stats.nba.com/stats/scoreboardV2?DayOffset=0&LeagueID=00&gameDate={month}%2F{day}%2F{year}".format(month=str(month_),day=str(day_), year=str(year))
      header = {'User-Agent': "test"}
      try:
        data = requests.get(url, headers = header).json()
        games = data["resultSets"][0]["rowSet"]
        for i in range(0, len(games)):
          game_id = games[i][2]
          if game_id[2] == '2' or game_id[2] == '4': # 2 reps regula season, 4 reps playoffs (1 reps preseason, 3 reps all star game)
            away_team_name = data["resultSets"][1]["rowSet"][i*2][5] + " " + data["resultSets"][1]["rowSet"][i*2][6]
            away_record = data["resultSets"][1]["rowSet"][i*2][7]
            home_team_name = data["resultSets"][1]["rowSet"][i*2 + 1][5] + " " + data["resultSets"][1]["rowSet"][i*2 + 1][6]
            home_record = data["resultSets"][1]["rowSet"][i*2 + 1][7]
            last_game_id = data["resultSets"][3]["rowSet"][i][1]
            away_score = data["resultSets"][1]["rowSet"][i*2][22]
            home_score = data["resultSets"][1]["rowSet"][i*2 + 1][22]
            result = str(away_score) + "-" + str(home_score)
            is_playoff_game = 1 if game_id[2] == '4' else 0
            winning_team = away_team_name if away_score > home_score else home_team_name
            game_ids[game_id] = (away_team_name, away_record, home_team_name, home_record, last_game_id, result, is_playoff_game, winning_team)
      except requests.exceptions.RequestException as e:
        continue
      except Exception as e:
        if data == "":
          print("passed", url)
        else:
          print("Jk, this day isn't valid")
        pass
df = pd.DataFrame(game_ids)
df_t = df.T
df_t.columns = ["away team", "away team record",  "home team", "home team record", "last game's id", "result(away-home)", "playoff game", "winning team"]
df_t

In [0]:
df = df_t
df.to_csv("games_2009_to_2019.csv")

In [0]:
from google.colab import files
files.download('games_2009_to_2019.csv')

# Preprocessing

In [0]:
df = pd.read_csv('/content/drive/My Drive/games_2009_to_2019.csv')

In [0]:
# df sometimes adds an extra indexing column and i'm not sure when lol
# this drops the extra column and renames the game_id column so it's consistent with the others
df = df.drop(columns=['Unnamed: 0'])
df = df.rename(columns={'Unnamed: 0.1': 'Unnamed: 0'})

In [0]:
# splits records into separate columns
df[['away team wins','away team losses']] = df["away team record"].str.split("-",expand=True).astype(int)
df['away team win pct'] = (df["away team wins"]) / ((df["away team wins"]) + (df["away team losses"]))
df[['home team wins','home team losses']] = df["home team record"].str.split("-",expand=True).astype(int)
df['home team win pct'] = (df["home team wins"]) / ((df["home team wins"]) + (df["home team losses"]))

# splits scores into separate columns
scores = df["result(away-home)"].str.split("-",expand=True).astype(int)
df['point differential'] = abs(scores[0] - scores[1])
df[['away points', 'home points']] = df["result(away-home)"].str.split("-",expand=True).astype(int)

In [0]:
# sets game_id as index (it's called "Unnamed: 0", idk why)
df.set_index("Unnamed: 0", inplace=True)
df.head()

In [0]:
# creates a list of winners of the last game, then adds it to the df
# does that by checking if last game id is in our df. If yes, get the winner, else return NA
list_ = []
id_list = df.index.tolist()
for i, row in df.iterrows():
  last_gm_id = np.int64(row[4])
  if last_gm_id in id_list:
    l_g_winner = df.loc[last_gm_id]['winning team']
    list_.append(l_g_winner)
  else:
    list_.append('NA')

df['last game winner'] = list_

# PPG
Calculating and adding PPG to df

In [0]:
# iterates through each row
# adds a teams points to a dict that keeps track of teams' ppg
# maintains running list of ppg at the time of each game
away_team_ppg = []
home_team_ppg = []


ppg_dict = {}
for i, row in df.iterrows():
  if str(i)[-4:] == "0001":
    ppg_dict = {}
    # print(str(i))
    print(row)

  if row[0] in ppg_dict:
    ppg_dict[row[0]][0] += df.loc[i]['away points']
    ppg_dict[row[0]][1] += 1.0
    ppg_dict[row[0]][2] = ppg_dict[row[0]][0] / ppg_dict[row[0]][1]
  else: 
    ppg_dict[row[0]] = [df.loc[i]['away points'], 1.0, df.loc[i]['away points']]
   
  away_team_ppg.append(ppg_dict[row[0]][2])

  if row[2] in ppg_dict:
    ppg_dict[row[2]][0] += df.loc[i]['home points']
    ppg_dict[row[2]][1] += 1.0
    ppg_dict[row[2]][2] = ppg_dict[row[2]][0] / ppg_dict[row[2]][1]
  else: 
    ppg_dict[row[2]] = [df.loc[i]['home points'], 1.0, df.loc[i]['home points']]
    
  home_team_ppg.append(ppg_dict[row[2]][2])
ppg_dict

# df['away team ppg'] = away_team_ppg
# df['home team ppg'] = home_team_ppg

# export csv

In [0]:
df.to_csv('games_2009_to_2019_updated_2.csv')
from google.colab import files
files.download('games_2009_to_2019_updated_2.csv')